# Mapping commute duration in the tri-state area
https://censusreporter.org/tables/B08303/

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import altair as alt

import cenpy as cen
import cenpy.tiger as tiger
import jenkspy

from getpass import getpass

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
CENSUS_API_KEY = getpass('Enter your Census API Key: ')

Enter your Census API Key: ········


## Commute duration

In [4]:
# Setting a variable (con) for making a call to this specific table for Median Household Income.
con = cen.remote.APIConnection('ACSDT5Y2020',apikey=CENSUS_API_KEY)

# Looking at the variables available in this table.
variables = con.variables

# Identify the columns to map. Included here is Median Household Income and total population
columns = [
    'B08303_001', 
    'B08303_002', 
    'B08303_003', 
    'B08303_004', 
    'B08303_005', 
    'B08303_006', 
    'B08303_007', 
    'B08303_008', 
    'B08303_009', 
    'B08303_010', 
    'B08303_011', 
    'B08303_012', 
    'B08303_013'
]

# Notice the E was absent from the columns identified. The E identifies the column as an estimate. But we also want to include margins of error with our estimate. Hence the code below.
columns_E = [i+'E' for i in columns]
# columns_M = [i+'M' for i in columns]

# Define the units of geography, as well as a filter for what we are looking for. 
# State codes can be found at https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html
# Can also use the Missouri Census Data Center, which has great resources for lookups - https://mcdc.missouri.edu/
# Geographies: zip code tabulation area, school district (elementary), school district (secondary), combined statistical area, congressional district, county, tract, block group
# Filters on the geography: filter to tracts within state and county = {'state':'12', 'county':'073'}
g_unit = 'county'
g_filter = {'state':'36'}

# Call the API for the tables/columns at the county level for the state of West Virginia and save to a dataframe
df_ny_raw = con.query(columns_E, geo_unit=g_unit, geo_filter=g_filter)

df_ny = df_ny_raw[df_ny_raw.county.isin(['005','047', '061', '081', '085', '119', '079', '087', '103', '059', '111', '027', '071'])].copy(deep=True)

df_ny_raw.head()

,B08303_001E,B08303_002E,B08303_003E,B08303_004E,B08303_005E,B08303_006E,B08303_007E,B08303_008E,B08303_009E,B08303_010E,B08303_011E,B08303_012E,B08303_013E,state,county
0,18308,1671,3392,2898,2201,2026,1044,1825,493,555,970,736,497,36,003
1,31039,2599,5881,5042,3812,3327,1725,2514,689,813,2274,1734,629,36,009
2,35006,1805,6113,7406,6593,4703,1864,2710,388,533,1403,752,736,36,015
3,25990,1796,3574,3134,3035,3500,1687,2494,1034,1042,2565,1316,813,36,021
4,132346,4123,12215,15775,18270,14957,7385,15952,3957,5736,11897,12035,10044,36,027


In [5]:
# NJ
g_unit = 'county'
g_filter = {'state':'34'}

df_nj_raw = con.query(columns_E, geo_unit=g_unit, geo_filter=g_filter)
df_nj_raw.head()

df_nj = df_nj_raw[df_nj_raw.county.isin(['003', '017', '031', '023', '025', '029', '035', '013', '027', '037', '019', '021', '039'])].copy(deep=True)

# county codes:
# https://www2.census.gov/geo/docs/reference/codes/files/st34_nj_cousub.txt
# 003 - Bergen County, NJ
# 017 - Hudson County, NJ
# 031 - Passaic County, NJ
# 023 - Middlesex County, NJ
# 025 - Monmouth County, NJ
# 029 - Ocean County, NJ
# 035 - Somerset County, NJ
# 013 - Essex County, NJ
# 039 - Union County, NJ
# 027 - Morris County, NJ
# 037 - Sussex County, NJ
# 019 - Hunterdon County, NJ
# 021 - Mercer County

df_nj.county.unique()

array(['003', '021', '027', '039', '013', '017', '019', '023', '025',
       '029', '031', '035', '037'], dtype=object)

In [6]:
# CT
g_unit = 'county'
g_filter = {'state':'09'}

df_ct_raw = con.query(columns_E, geo_unit=g_unit, geo_filter=g_filter)
df_ct_raw.head()

df_ct = df_ct_raw[df_ct_raw.county.isin(['009','001'])].copy(deep=True)

# https://www2.census.gov/geo/docs/reference/codes/files/st09_ct_cousub.txt
# 009 - New Haven County, Connecticut
# 001 - Fairfield County

df_ct.county.unique()

array(['009', '001'], dtype=object)

In [7]:
# PA
g_unit = 'county'
g_filter = {'state':'42'}

df_pa_raw = con.query(columns_E, geo_unit=g_unit, geo_filter=g_filter)
df_pa_raw.head()

df_pa = df_pa_raw[df_pa_raw.county.isin(['103','089'])].copy(deep=True)

# https://www2.census.gov/geo/docs/reference/codes/files/st42_pa_cousub.txt
# 103 - Pike County, PA
# 089 - Monroe County, Pennsylvania

df_pa.county.unique()

array(['089', '103'], dtype=object)

In [8]:
# concat all
df = pd.concat([df_ny, df_nj, df_ct, df_pa])

In [9]:
# rename columns - adding num_ to the beginning of each variable so that the column names can be called in JS
df.rename(columns = 
          {
    'B08303_001E': 'total', 
    'B08303_002E': 'num_less_than_5', 
    'B08303_003E': 'num_5_to_9', 
    'B08303_004E': 'num_10_to_14', 
    'B08303_005E': 'num_15_to_19', 
    'B08303_006E': 'num_20_to_24', 
    'B08303_007E': 'num_25_to_29', 
    'B08303_008E': 'num_30_to_34', 
    'B08303_009E': 'num_35_to_39', 
    'B08303_010E': 'num_40_to_44', 
    'B08303_011E': 'num_45_to_59', 
    'B08303_012E': 'num_60_to_89', 
    'B08303_013E': 'num_over_90'
          }, 
          inplace = True)

In [10]:
# convert to int
df['total'] = df['total'].astype(int)
df['num_less_than_5'] = df['num_less_than_5'].astype(int)
df['num_5_to_9'] = df['num_5_to_9'].astype(int)
df['num_10_to_14'] = df['num_10_to_14'].astype(int)
df['num_15_to_19'] = df['num_15_to_19'].astype(int)
df['num_20_to_24'] = df['num_20_to_24'].astype(int)
df['num_25_to_29'] = df['num_25_to_29'].astype(int)
df['num_30_to_34'] = df['num_30_to_34'].astype(int)
df['num_35_to_39'] = df['num_35_to_39'].astype(int)
df['num_40_to_44'] = df['num_40_to_44'].astype(int)
df['num_45_to_59'] = df['num_45_to_59'].astype(int)
df['num_60_to_89'] = df['num_60_to_89'].astype(int)
df['num_over_90'] = df['num_over_90'].astype(int)

In [11]:
# percent of each borough that reported a commute of that length 
# adding in front of each num_ariable 
df['pct_less_5'] = (df['num_less_than_5']/df['total'])*100
df['pct_5_to_9'] = (df['num_5_to_9']/df['total'])*100
df['pct_10_to_14'] = (df['num_10_to_14']/df['total'])*100 
df['pct_15_to_19'] =  (df['num_15_to_19']/df['total'])*100
df['pct_20_to_24'] = (df['num_20_to_24']/df['total'])*100
df['pct_25_to_29'] = (df['num_25_to_29']/df['total'])*100
df['pct_30_to_34'] = (df['num_30_to_34']/df['total'])*100
df['pct_35_to_39'] = (df['num_35_to_39']/df['total'])*100
df['pct_40_to_44'] = (df['num_40_to_44']/df['total'])*100
df['pct_45_to_59'] = (df['num_45_to_59']/df['total'])*100
df['pct_60_to_89'] = (df['num_60_to_89']/df['total'])*100
df['pct_over_90'] = (df['num_over_90']/df['total'])*100

In [12]:
# create column for what the majority of people's commutes is
df['plurality_time'] = df[['pct_less_5', 'pct_5_to_9', 'pct_10_to_14', 'pct_15_to_19', 'pct_20_to_24',
       'pct_25_to_29', 'pct_30_to_34', 'pct_35_to_39', 'pct_40_to_44',
       'pct_45_to_59', 'pct_60_to_89', 'pct_over_90']].idxmax(axis=1)

In [13]:
# clean column so it's formatted without the underscores, Vs and pct suffix that were required to format the var name
df['plurality_time'] = df.plurality_time.str.replace('pct_less_5', '<5')
df['plurality_time'] = df.plurality_time.str.replace('pct_over_90', '>90')
df['plurality_time'] = df.plurality_time.str.replace('pct_', '')
df['plurality_time'] = df.plurality_time.str.replace('_', ' ')

In [14]:
# create column for the actual percentage for the above category
df['plurality_time_pct'] = df[['pct_less_5', 'pct_5_to_9', 'pct_10_to_14', 'pct_15_to_19', 'pct_20_to_24',
       'pct_25_to_29', 'pct_30_to_34', 'pct_35_to_39', 'pct_40_to_44',
       'pct_45_to_59', 'pct_60_to_89', 'pct_over_90']].max(axis=1)

In [15]:
# save to csv
# df.to_csv('ny-area-commute-duration-2020.csv', index=False)

## Create geojson

In [16]:
# read in census data
counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_county_5m.zip')

In [17]:
# create a GEOID column for merge
df['GEOID'] = df['state'] + df['county']

In [18]:
print(len(df))
print(len(counties))

30
3233


In [19]:
# merge data
df_merged = df.merge(counties, on='GEOID', how='left')

In [20]:
len(df_merged)

30

In [21]:
# using epsg 2263 for NY/Long Island
df_geodata = gpd.GeoDataFrame(data=df_merged, geometry=df_merged['geometry'], crs='epsg:2263')

In [22]:
# save to geojson
with open('ny-area-commute-duration-2020.geojson', 'w') as f:
        f.write(df_geodata.to_json())

In [23]:
df_geodata.columns

Index(['total', 'num_less_than_5', 'num_5_to_9', 'num_10_to_14',
       'num_15_to_19', 'num_20_to_24', 'num_25_to_29', 'num_30_to_34',
       'num_35_to_39', 'num_40_to_44', 'num_45_to_59', 'num_60_to_89',
       'num_over_90', 'state', 'county', 'pct_less_5', 'pct_5_to_9',
       'pct_10_to_14', 'pct_15_to_19', 'pct_20_to_24', 'pct_25_to_29',
       'pct_30_to_34', 'pct_35_to_39', 'pct_40_to_44', 'pct_45_to_59',
       'pct_60_to_89', 'pct_over_90', 'plurality_time', 'plurality_time_pct',
       'GEOID', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry'],
      dtype='object')